#### Demo 3 - Hadoop Shuffle & TOS
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2019`__

Designing MapReduce algorithms involves two kinds of planning. First we have to figure out which parts of a calculation can be performed in parallel and which can't. Then we have to figure out how to put those pieces together so that the right information ends up in the right place at the right time. That's what the Hadoop shuffle is all about. Today we'll talk about a few techniques to optimize your Hadoop jobs. By the end of this demo you should be able to:  
* ... __identify__ what makes the Hadoop Shuffle potentially costly.
* ... __define__ local aggregation & combiners.
* ... __implement__ partial, unordered, and total order sort.
* ... __create__ custom counters for your Hadoop Jobs.
* ... __describe__ the order inversion pattern & when to use it (ie: relative frequencies).

**Note**: Hadoop Streaming syntax is very particular. Make sure to test your python scripts before passing them to the Hadoop job and pay careful attention to the order in which Hadoop job parameters are specified.

In [1]:
!hadoop version

Hadoop 2.6.0-cdh5.16.2
Subversion http://github.com/cloudera/hadoop -r 4f94d60caa4cbb9af0709a2fd96dc3861af9cf20
Compiled by jenkins on 2019-06-03T10:43Z
Compiled with protoc 2.5.0
From source with checksum 79b9b24a29c6358b53597c3b49575e37
This command was run using /usr/lib/hadoop/hadoop-common-2.6.0-cdh5.16.2.jar


## Hadoop Streaming Docs
https://hadoop.apache.org/docs/r2.6.0/hadoop-mapreduce-client/hadoop-mapreduce-client-core/HadoopStreaming.html   


### Notebook Set-Up

In [2]:
# imports & magic commands
import sys
%reload_ext autoreload
%autoreload 2

In [48]:
# globals
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HOME_DIR = "/media/notebooks" # this is where docker mounts your repo, ADJUST AS NEEDED
DEMO_DIR = HOME_DIR + "/LiveSessionMaterials/wk03Demo_HadoopShuffle"
HDFS_DIR = "/user/root/demo3"
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/demo3': File exists


In [49]:
# <--- SOLUTION --->
# ... for instructors ...
HOME_DIR = "/media/notebooks/Instructors"
DEMO_DIR = HOME_DIR + "/LiveSessionMaterials/wk03Demo_HadoopShuffle/master"

In [50]:
# get path to notebook
PWD = !pwd
PWD = PWD[0]

In [51]:
# store notebook environment path
from os import environ
PATH  = environ['PATH']

__`REMINDER:`__ If you are running this notebook from the course Docker container you can track your Hadoop Jobs using the UI at: http://localhost:19888/jobhistory/

### Load the Data

In this notebook, we'll continue working with the  _Alice in Wonderland_ text file from HW1 and the test file we created for debugging. Run the following cell to confirm that you have access to these files and save their location to a global variable to use in your Hadoop Streaming jobs. 

In [52]:
# make a data subfolder - RUN THIS CELL AS IS
!mkdir -p data

In [53]:
# (Re)Download Alice Full text from Project Gutenberg - RUN THIS CELL AS IS 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   174k      0 --:--:-- --:--:-- --:--:--  174k


In [54]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [55]:
# save the paths - RUN THIS CELL AS IS (if Option 1 failed)
ALICE_TXT = PWD + "/data/alice.txt"
TEST_TXT = PWD + "/data/alice_test.txt"

In [56]:
# confirm the files are there - RUN THIS CELL AS IS
!echo "######### alice.txt #########"
!head -n 6 {ALICE_TXT}
!echo "######### alice_test.txt #########"
!cat {TEST_TXT}

######### alice.txt #########
﻿The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
######### alice_test.txt #########
This is a small test file. This file is for a test.
This small test file has two small lines.


### Load the file into HDFS for easy access

See the docs for the difference between `-put` and `-copyFromLocal`:     

https://hadoop.apache.org/docs/r2.6.0/hadoop-project-dist/hadoop-common/FileSystemShell.html


In [57]:
# load the input files into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -put {TEST_TXT} {ALICE_TXT} {HDFS_DIR}

put: `/user/root/demo3/alice_test.txt': File exists
put: `/user/root/demo3/alice.txt': File exists


# Content Overview: Hadoop Shuffle

The week 3 reading from Chapter 3 of _Data Intensive Text Processing with Map Reduce_ by Lin and Dyer discusses 5 key techniques for controlling execution and managing the flow of data in MapReduce:
1. using complex data structures to communicate partial results
2. user-specified initialization/termination code before/after each map/reduce task
3. preserving state across multiple keys
4. controling the sort order of intermediate keys
5. specifying the partitioning of the key space

### Our goal in employing these techniques is to minimize the amount we have to move the data. This involves keeping track of what data is stored where at each stage in our job (_DITP figure 2.5 and 2.6_):
*Recall that a Hadoop cluster stores our data on datanotes and ships the programmer's map and reduce code to those nodes to perform those transformations in place.*

![HDFSdiagram](DITP_fig2-5,6.png)


### Though it isn't always possible to do so, ideally we'd like to design an implementation that retrieves the result in a single MapReduce job (_DITP figure 2.4_):
![MRdiagram](DITP_fig2-4.png)

### Shuffle & Sort Detail (_Hadoop, The Definitve Guide, by Tom White; fig 7-4_):   
![CircularBuffer-DFG](HDG_fig7-4-annotated.png)


> __DISCUSSION QUESTIONS:__  
> * What work does your Hadoop cluster have to do at the shuffle stage? 
> * What determines the time complexity of this work? 
> * Compare the Lyn & Dyer diagram with the om White diagram. How might the Lyn & Dyer diagram be misleading?
> * What is a combiner, how does it impact the shuffle? 'where' does the combining happen?
> * What is local aggregation? Is a combiner the only way to do it?
> * What is a partitioner and how does it impact the shuffle? How/where can we specify custom paritioning behavior? If we don't specify a partitioner, will Hadoop still partition the data? How?
> * Can you think of an example of a task that can't be accomplished in a single MapReduce job? Explain.

#### *_A note on the Lin and Dyer reading (and other readings you may encounter):_

>In MapReduce, the programmer defines a mapper and a reducer with the following signatures:   
map: (k1; v1) -> [(k2; v2)]   
reduce: (k2; [v2]) -> [(k3; v3)]   
where [..] denotes a list

_It is important to note that the MapReduce framework ensures that the keys are ordered, so we know that if a key is different from the previous one, we have moved into a new key group. In contrast to the Java API, where you are provided an iterator over each key group (as per above Lin and Dyer version), in Streaming you have to find key group boundaries in your program. (Page 39, Hadoop - The Definitive Guide)_

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS__  
* What work does your Hadoop cluster have to do at the shuffle stage? What determines the time complexity of this work?
> Short digression: there is something slightly deceptive about this kind of MapReduce diagram -- the arrows make it look like data is flowing from step to step down the chart. Of course that is not the case physically. The shuffle stage is where the data gets moved. It doesn't linearly follow the 'paritioning' but includes all the work done by the framework after the mapper emits the data and before the reducers start. The work involved is costly in two ways: first in order to perform the sort Hadoop has to look at which keys are present on each data node & make a bunch of comparisions to sort these keys and plan where records with those keys are going to end up, then the data has to be transfered. The complexity is a function of the number of records output by our mappers (NOTE: this is different than the number of records we started with.)

* Compare the Lyn & Dyer diagram with the om White diagram. How might the Lyn & Dyer diagram be misleading?
> In the Lyn & Dyer diagram combiners appear to only be executed at the map phase.

* What is a combiner, how does it impact the shuffle? 'where' does the combining happen?
> A combiner is an aggregation script specfied by the programmer that will take mapper output records with the same key and turn them into a single, combined, record. Usually this is desireable because fewer records = a faster shuffle. However its important to note that Hadoop uses the combiners strategically -- sometimes records are combined before leaving the mapper node, sometimes after arriving at a reducer node (this is related to the 'circular buffer' that you may remember discussed in the async). There are two key takeaways here 1) that Hadoop doesn't guarantee that it will perform the combining on any or all records and 2) Hadoop makes smart choice about when to hold data in memory/combine/spill to disk... saving a lot of work for the programmer.

* What is local aggregation? Is a combiner the only way to do it?
> Local aggregation is basically what we described a minute ago: reducing the number of records that need to be transfered over the network in the shuffle state _OR_ between two sequential MapReduce jobs. Combiners are _NOT_ the only way to perform local aggregation, we could also use an in memory form of aggregation eg. a python dicitonary. Of course there are risks to this approach if your mappers are emitting more records than the ones they read in. (_NOTE:_ local aggregation is one example of the 2nd technique that Lin & Dyer list: "preserving state across multiple keys").


* What is a partitioner and how does it impact the shuffle? How/where can we specify custom partitioning behavior? If we don't specify a partitioner, will Hadoop still partition the data? How?
> The 'partitioner' parameter in our Hadoop jobs (we saw this last week in live session) simply tells Hadoop how to determine which keys end up together on the same reducer node. Custom paritioning is just a matter of manipulating the key format/data structure -- this would happen in a  mapper script there is no 'paritioner' script. Anytime the programmer specifies more than one reducer then Hadoop will perform partitioning... if we don't tell it how to partittion explicitly then Hadoop will make its own decision about how to split up the key space.

* Can you think of an example of a task that can't be accomplished in a single MapReduce job? Explain.
> Students should be able to come up with sorted word count (challenge question from the end of live session in week2) --> can't sort until after reducing so you need a second job. We'll look at an example below of calculating relative frequencies... it would be good to guide them to consider the key challenge in that task: you can't divide until you have a total but what if you don't want to hold all of the counts in memory? (we'll teach order inversion below, no need to mention that here).

# Exercise 1: Relative Frequencies
In last week's live session and HW1 we used Word Count as a cannonical example of an embarassingly parallel task. At first glance, computing relative frequencies (`word count / total count`) seems like it would be just as easy to implement -- after all it's just word count with an extra division at the end. However this task actually presents a small design challenge that is perfect to illustrate a few of the techniques that Lin & Dyer talk about.

__DISCUSSION:__
> * Talk through what the MapReduce job would look like? What is the challenge here?
> * Is it possible to compute relative frequencies in a single MapReduce job? 
> * Is it possible to compute relative frequencies in a single MapReduce job with multiple reducers?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before exercise 1)__  
* Talk through what the MapReduce job would look like? What is the challenge here?  
> The map would work the same as in word count, then the reducer would aggregate counts and also add up a total word count so that it can divide each word's count by the total. The challenge is that we won't know the total until after reducing. In word count we count in parallel and then add at the end. To compute frequencies we can still count in parallel but then we need to add both individual words and the total number of words before we can divide. 

* Is it possible to compute relative frequencies in a single MapReduce job?  
> Yes, we could hold all the word counts in memory and then divide at the end... but that's not scalable. 

* Is it possible to compute relative frequencies in a single MapReduce job with multiple reducers?
> No, if we use multiple reducers we'd have no way to get the total-- we'd just have partial totals on each reducer node. We'd need a second job to get the full total & perform the division.

.... OK _actually_ there IS a scalable a way to do this. Its called the order inversion pattern. Lets take a look:

### Exercise 1 Tasks:

* __a) read provided code:__ We've provided a naive interpretation to get you started. Take a look at __`Frequencies/mapper.py`__, __`Frequencies/combiner.py`__  and __`Frequencies/reducer.py`__.  

* __b) discuss:__ How does it resolve the challenge of computing the total? Uncomment line 22 in the mapper and lines 26-27 in the reducer to take advantage of this 'solution', then run the unit tests below, to confirm that you understand what's going on in each of these scripts. Despite solving the problem of computing the total in a single map-reduce job, what is wrong with this approach? 

* __c) fix the problem:__ To fix the problem, all we need to do is make a small change to the key used to emit the total counts (you need to do this in both the mapper and reducer). Think about Hadoop's default sorting. What keys arrive first? What key could you assign to the total that would be guaranteed to arrive first? Once you are satisfied that your solution works, run the provided Hadoop job on the test file & then the full `alice` text. (note it has a single reducer for now).  

* __d) discuss:__  Now, re-run the job with 4 reducers, what happens to the results? What would we have to do to fix the problem?

In [13]:
# part b - make sure scripts are executable (RUN THIS CELL AS IS)
!chmod a+x Frequencies/mapper.py
!chmod a+x Frequencies/combiner.py
!chmod a+x Frequencies/reducer.py

In [14]:
# part b - unit test mapper script
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py

foo	1
!total	1
foo	1
!total	1
quux	1
!total	1
labs	1
!total	1
foo	1
!total	1
bar	1
!total	1
quux	1
!total	1


In [15]:
# part b - unit test map-combine (sort mimics shuffle) (RUN THIS CELL AS IS)
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py | sort -k1,1 | Frequencies/combiner.py

!total	7
bar	1
foo	3
labs	1
quux	2


In [16]:
# part b - unit test map-combine-reduce (sort mimics shuffle) (RUN THIS CELL AS IS)
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py | sort -k1,1 | Frequencies/combiner.py | Frequencies/reducer.py 

!total	1.0
bar	0.14285714285714285
foo	0.42857142857142855
labs	0.14285714285714285
quux	0.2857142857142857


In [17]:
# parts c - clear the output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/frequencies-output
# NOTE: this directory won't exist unless you are re-running a job, that's fine.

rm: `/user/root/demo3/frequencies-output': No such file or directory


In [18]:
# parts c - Hadoop streaming job (RUN THIS CELL AS IS FIRST, then make your modification)
!hadoop jar {JAR_FILE} \
  -files Frequencies/reducer.py,Frequencies/mapper.py,Frequencies/combiner.py \
  -mapper mapper.py \
  -combiner combiner.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice_test.txt \
  -output {HDFS_DIR}/frequencies-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob711175312799087886.jar tmpDir=null
21/01/18 11:00:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:00:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:00:50 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/18 11:00:51 INFO mapreduce.JobSubmitter: number of splits:2
21/01/18 11:00:51 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610967571082_0001
21/01/18 11:00:51 INFO impl.YarnClientImpl: Submitted application application_1610967571082_0001
21/01/18 11:00:51 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610967571082_0001/
21/01/18 11:00:51 INFO mapreduce.Job: Running job: job_1610967571082_0001
21/01/18 11:00:58 INFO mapreduce.Job: Job job_1610967571082_0001 running in uber mode : false
21/01/18 11:00:58 INFO mapreduce.Job:  map 0% reduce 0%
21/01/

In [19]:
# take a look at the first few results (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/frequencies-output/part-00000 | head -n 10

!total	1.0
a	0.1
file	0.15
for	0.05
has	0.05
is	0.1
lines	0.05
small	0.15
test	0.15
this	0.15


In [20]:
# <--- SOLUTION --->
# parts c - Hadoop streaming job (RUN THIS CELL AS IS FIRST, then make your modification)
!hdfs dfs -rm -r {HDFS_DIR}/frequencies-output
!hadoop jar {JAR_FILE} \
  -files Frequencies/reducer.py,Frequencies/mapper.py,Frequencies/combiner.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice.txt \
  -output {HDFS_DIR}/frequencies-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1
!hdfs dfs -cat {HDFS_DIR}/frequencies-output/part-00000 | head -n 10

Deleted /user/root/demo3/frequencies-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8811832056850768385.jar tmpDir=null
21/01/18 11:01:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:01:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:01:16 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/18 11:01:16 INFO mapreduce.JobSubmitter: number of splits:2
21/01/18 11:01:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610967571082_0002
21/01/18 11:01:16 INFO impl.YarnClientImpl: Submitted application application_1610967571082_0002
21/01/18 11:01:16 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610967571082_0002/
21/01/18 11:01:16 INFO mapreduce.Job: Running job: job_1610967571082_0002
21/01/18 11:01:21 INFO mapreduce.Job: Job job_1610967571082_0002 running in uber mode : false
21/01/18 11:01:21

__Expected Results:__

<table>
<th>part c (test)</th>
<th>part c (full)</th>
<tr>
<td><pre>
!total	1.0
a	0.1
file	0.15
for	0.05
has	0.05
is	0.1
lines	0.05
small	0.15
test	0.15
this	0.15
</pre></td>
<td><pre>
!total	1.0
a	0.022728759238668322
abide	6.540650140623977e-05
able	3.270325070311989e-05
about	0.003335731571718229
above	9.810975210935967e-05
absence	3.270325070311989e-05
absurd	6.540650140623977e-05
accept	3.270325070311989e-05
acceptance	3.270325070311989e-05
</pre></td>
</tr></table>


__DISCUSSION:__  
> *  Was the original implementation scalable? How does it resolve the challenge of computing the total? 
> * Despite solving that problem, what is wrong with this approach?  
> * Did you come up with a new key that is guaranteed to arrive first? how?
> * What happened when you went from 1 reducer to 4, why? (HINT: Use the UI to see the error logs for the failed reduce tasks)
> * What do we need to do with the total counts when we move up to 4 reducers?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after exercise 1)__  
*  Was the original implementation scalable? How does it resolve the challenge of computing the total? 
  > Yes, instead of holding records in memory it emits a second record allowing us to count the totals without
  relying on the word counts.
  
* Despite solving that problem, what is wrong with this approach?  
  > The Hadoop Shuffle ensures that the key "Total" arrives in alphabetical order... after most of our word counts have already been added up. We need the total to arrive first.

* Did you come up with a new key that is guaranteed to arrive first? how?
  > *Total, !Total etc
  
* What happened when you went from 1 reducer to 4, why?
  > The Hadoop job fails due to a zero division error (ask them if they used the UI to see the error logs for the failed reduce tasks). This happens because the total key only gets sent to one of the 4 reducers. 
  
* What do we need to do with the total counts when we move up to 4 reducers?
  > We need a way to make the total available to each reducer node (partition)

# Exercise 2: Custom Partitioning

Last week in Breakout 4 you learned out to implement a secondary sort -- that is, you learned how to tell Hadoop to order key-value pairs within each partition based on the value. However you also saw the limitation of this simple secondary sort: namely that sorting within a partition is not very useful if you need to use multiple reducers because, for example, the top word could end up in any one of the partitions and the next highest might end up in a totally different partition. Of course post processing your partially sorted partition files (eg. using mergesort) might solve this problem, but if your data is too large to fit on a single machine that is not a viable solution.

Luckily, Hadoop provides a way to partition the data across reducers in a user-defined way.  This is done telling Hadoop to use all or part of the composite key as a partitioning key. All lines with the same partition key are guaranteed to be processed by the same reducer. This is similar to the sort key, but allows for control at a higher level. This "custom partitioning" will both solve our sorting troubles from last week and solve the problem you saw when using multiple reducers in Exercise 1 this week. To use custom partitioning, there are 2 more parameters we need to add to our Hadoop Streaming Command:

 __`-D mapreduce.partition.keypartitioner.options="-k1,1"`__: tells Hadoop that we want to use the first key field as a partition key. Note: just like the `keycomparator`, `keypartitioner` must be used in conjuction with `stream.num.map.output.key.fields`.

__`-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner`__: tells Hadoop that we want to partition the data in a custom way. Note about partitioner: this line **MUST** appear after all of the `-D` options or it will be ignored.

> __DISCUSSION QUESTIONS (before exercise 2):__
* Quick review: What is a composite key? 
* Quick review: Practically speaking, what is a 'partition'? How is this concept related to the HDFS output of a job?
* What is the difference between the partition key & the 'sort' key?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before exercise 2)__
* Quick review: What is a composite key?  
> We created a composite key in Breakout 4 -- that was when we told Hadoop to treat the first two fields together as the key. We did this above for sorting purposes. Another way to create a composite key would have been to simply make a string joinin the two fields with a comma or dash (anything other than the Hadoop field delimiter).

* Quick review: Practically speaking, what is a 'partition'? How is this concept related to the HDFS output of a job?
> Partition = data split / guaranteed co-location of records for processing in reduce tasks. Number of partitions = number of reduce tasks = number of files in the HDFS output directory.

* What is the difference between the partition key & the 'sort' key?
> Normally Hadoop's only guarantee is that records with the same key get processed on the same reducer node. However, if we wanted to process multiple keys together on a given reducer, we could use a partition key which determines which reducer note handles a record... but still allows us to sort on something more granular (eg. another field).

### Exercise 2 Tasks:

* __a) read provided code:__ In __`PartitionSort/mapper.py`__ we've provided a function that will assign a custom partition key (just a letter) to each word. We're going to use this mapper to sort our sample file with 3 partitions. Read this script. 

* __b) discus:__ How does the mapper decide which partition to assign each record? When you print out the results in what order do you expect to see the records?

* __c) Hadoop job:__ Add the required parameters to complete the Hadoop Streaming code below. Your job should partition based on the newly added first key field, and sort alphabetically by word. Run your job. [__`Hint:`__ Don't forget to specify the number of fields!]

* __d) discuss:__ Examine the output from your job. Compare it to the partitioning function we used. Are the words sorted alphabetically? What is suprising about the partition key that ended up in `part-00000`?

* __e) code:__ If time permits, modify your job so that it sorts the words by _count_ instead (still using 3 partitions). To do this you will need to change the partition function in __`PartitionSort/mapper.py`__ so that it partitions based on count instead of the first letter of the word. Use 4 and 8 as your cut-points. You will also need to modify one of the Hadoop parameters (which one? why?). Run your job. Are you able to get a total sort? Why/why not?

In [21]:
%%writefile PartitionSort/sample.txt
foo	5
quux	9
labs	100
bar	5
qi	1

Writing PartitionSort/sample.txt


In [22]:
# load sample file into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal PartitionSort/sample.txt {HDFS_DIR}

In [23]:
# part a - complete your work above then RUN THIS CELL AS IS
!chmod a+x PartitionSort/mapper.py

In [24]:
# parts a - clear output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/psort-output

rm: `/user/root/demo3/psort-output': No such file or directory


In [47]:
# part a - Hadoop streaming command - ADD SORT and PARTITION PARAMETERS HERE
!hadoop jar {JAR_FILE} \




  -files PartitionSort/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \

  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/psort-output \
  -cmdenv PATH={PATH}\
  -numReduceTasks 3

In [26]:
# <--- SOLUTION --->
# part a - Hadoop streaming command
!hdfs dfs -rm -r {HDFS_DIR}/psort-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k1,1"  \
  -files PartitionSort/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/psort-output \
  -cmdenv PATH={PATH}\
  -numReduceTasks 3

rm: `/user/root/demo3/psort-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob729108224399404002.jar tmpDir=null
21/01/18 11:02:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:02:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/18 11:02:55 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/18 11:02:55 INFO mapreduce.JobSubmitter: number of splits:2
21/01/18 11:02:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610967571082_0003
21/01/18 11:02:55 INFO impl.YarnClientImpl: Submitted application application_1610967571082_0003
21/01/18 11:02:55 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610967571082_0003/
21/01/18 11:02:55 INFO mapreduce.Job: Running job: job_1610967571082_0003
21/01/18 11:03:01 INFO mapreduce.Job: Job job_1610967571082_0003 running in uber mode : false

In [27]:
# part a - Save results locally (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-0000* > PartitionSort/results.txt

In [28]:
# part a - view results (RUN THIS CELL AS IS)
!head PartitionSort/results.txt

B	labs	100	
B	quux	9	
C	bar	5	
C	foo	5	
A	qi	1	


In [29]:
# part a - look at first partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00000

B	labs	100	
B	quux	9	


In [30]:
# part a - look at second partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00001

C	bar	5	
C	foo	5	


In [31]:
# part a - look at third partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00002

A	qi	1	


__Expected Output:__

<table>
<th>part c</th>
<th>part e</th>
<tr>
<td><pre>
B	labs	100
C	qi	1
C	quux	9
A	bar	5
A	foo	5
</pre></td>
<td><pre>
B	labs	100
B	quux	9
C	foo	5
C	bar	5
A	qi	1
</pre></td>
</tr></table>

> __DISCUSSION QUESTIONS (exercise 2 debrief):__
* In the provided implementation how did we assign records to partitions?
* In part c, why didn't this partitioning result in alphabetically sorted words?
* Given what you saw in part c, how did you 'trick' Hadoop into doing a full sort (by count) in part e?
* If you didn't achive the full sort in `e` why not? On a larger data set what postprocessing would you have to do in this kind of scenario? Is this postprocessing non-trivial?
* In addition to changing the partition function what other Hadoop parameter did you have to change for part `e`?
* In the real world you wouldn't want your partition key as part of your output. What would we do to avoid this?
* Does anyone need any additional clarification about any of the Hadoop Streaming?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (exercise 2 debrief)__

* In the provided implementation how did we assign records to partitions?
 > according to the first letter in the word... a-h when in partition 'A', j-o in partition 'B' and p=z in partition 'C'
* In part c, why didn't this partitioning result in alphabetically sorted words?
 > We assigned the top(alphabetical) words to partition A, but when Hadoop concatenated the files, it put partition A last. This is not by chance. Hadoop uses a hash function to assign our human readible partition keys to an ordered list of partitions. The ordering that is logical to us, may or may not get preserved in the process. However, its worth noting that this hash function is consistent. i.e. we just say Hadoop order B-C-A, that means it will always order those three keys that way. We can take advantage of this fact to perform a Total Order Sort -- i.e. a sort with multiple paritions that doestn' require any post-processing. Implementing this will be one of the most important question on HW2.
 
* Given what you saw in part c, how did you 'trick' Hadoop into doing a full sort (by count) in part e?
> Since we knew partition B was going to go first, we put the highest counts there, followed by the 5-9 count words in 'C' and the less than 5 count words in 'A'.

* If you didn't achive the full sort in `e` why not? On a larger data set what postprocessing would you have to do in this kind of scenario? Is this postprocessing non-trivial?
> Not knowing about the hash function you may have simply assigned the top counts to A.. in that case we still have the counts segemented by top/middle/bottom we'd just need to figure out in what order the partitions should be concatenated. This is a lot less work than the merge-sort we needed for postprocessing our secondary sort in break out 4... but with a really large corpus this could still be in-ideal because it requires checking and comparing each partition.

* In addition to changing the partition function what other Hadoop parameter did you have to change for part `e`?
> the `keycomparator` -- so that it reverse numerical sorts on the 3rd field within each partition

* Does anyone need any additional clarification about any of the Hadoop Streaming parameters -- you will need to be fluent in these for HW2.

# Exercise 3: Relative Frequencies Revisited
### (+ intro to counters)

Now that you know how to use custom partition keys, let's use this techinique to fix the problem from part d in Exercise 1.

First a brief digression... A common challenge when implementing custom partitioning at scale is load balancing: _how can we ensure that each reducer node is doing approximately the same amount of work?_ Hadoop's option to define custom counters can be a useful way to monitor this kind of detail. Just like the built in  counters that you learned about last week, custom counters are a slight departure from statelessness. Normally we wouldn't want to share a mutable variable across multiple nodes, however in this case the framework manages so that you can increment them from any node without causing a race condition. 

To use a custom counter you'll just write an appropriately formatted line to the standard error stream. For example the following line would increment a counter called 'counter-name' in group 'MyWordCounters' by 10:
 > `sys.stderr.write("reporter:counter:MyWordCounters,counter-name,10\n")`
 
This line can be added to your mapper, combiner or reducer scripts and wraped in `if` clauses to increment only under certain conditions. If a counter with that name/group doesn't exsit yet the framework will create one. The values of your custom counters will be printed to the console in their respective groups just like the built in Job Tracking counters. Counters can only be incremented using integers (ie, floats are not supported).

Ok, armed with this new tool let's return to the relative frequencies task.

__DISCUSSION:__
> * Remember the problem we encountered at the end of Exercise 1? How would a custom partition key help us make sure the total counts get sent to each reducer node?
> * What does this do to the number of records being shuffled? Is there a more and a less efficient way to implement it from the perspective of amount of data shuffled?
> * Where would you implement the custom partition key?
> * How would you partition the records? (i.e what criteria would you use to assign keys)

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before exercise 2)__  

* How would a custom partition key help us make sure the total counts get sent to each reducer node?
> If each partition has a specfic key then we can explicitly send the totals to each one.

* What does this do to the number of records being shuffled?
> We will be emitting  alot more records - x4 for four reducers. But if we are using local aggregation this needn't increase the number of records being shuffled unduly. This would be a great situaion in which to use an in-mapper aggregator.

* Where would you implement the custom partition key?
> In the first mapper.

* How would you partition the records? (i.e what criteria would you use to assign keys)
> It doesn't really matter since we don't actually care where the records end up... however it would be smart to try and balance out the amount of work done on each reducer so in this case we could use the alphabet or some other EDA to write a partition function based on the words. Since we're going to use 4 reducers maybe you'd do something like a-g, h-n, o-t,u-z.

### Exercise 3 Tasks:

## UPDATE Jan 23, 2021 - Combiner has been removed from this exercise. Combiners do not play nice with secondary sort. https://issues.apache.org/jira/browse/MAPREDUCE-3310

* __a) add a custom partitioner:__  In __`MultiPart/mapper.py`__, and __`MultiPart/reducer.py`__ we've copied the base code from the frequencies job. Complete the code in this mapper so that it adds a custom partition key to each record and emits the total subcounts _once for each partition_. Assume we'll be using 4 partitions. Then make any required adjustments to your reducer and combiner to accomodate the new record format (your final output should not include the partition key, though you may want it there initially for debugging purposes).

* __b) discuss:__ Keep in mind that each partition still needs the total counts to arrive before the individual word counts. However since you've added a custom partition key to each record, the order inversion trick of adding a special character isn't going to work with Hadoop's default sorting behavior any more. Why not? What will you need to specify in your Hadoop job to make sure that the totals still arrive first?

* __c) unit test & run your job:__ Write a few unit tests to debug your scripts. When you have them working as expected run the Hadoop job. [__`NOTE`__ We've provided a few tests to get you started but you'll need to add a unix sort to mimic the sorting you discussed in part 'b'].

* __d) custom counters:__ Add custom counter(s) to your reducer code so that you can count how many records are processed by each reduce task. [__`TIP`__ use the partition key as the counter name and python3 string formatting to do this efficiently].

* __e) discuss:__ Rerun your Hadoop job and take a look at the custom counter values. What do they tell you? Are these counts the same as the number of keys in the result of each partition? Try changing the partitioning criteria in your mapper... what partitioning results in a really uneven split? what partitioning results in the most even split?

In [32]:
# part c - make sure scripts are executable (RUN THIS CELL AS IS)
!chmod a+x MultiPart/mapper.py
!chmod a+x MultiPart/reducer.py

In [33]:
# part c - unit test mapper script
!echo "foo foo quux labs foo bar quux" | MultiPart/mapper.py

A	foo	1
A	foo	1
C	quux	1
B	labs	1
A	foo	1
A	bar	1
C	quux	1
A	!total	7
B	!total	7
C	!total	7
D	!total	7


In [61]:
# part c - unit test map-combine-reduce (ADJUST SORT AS NEEDED)
!echo "foo foo quux labs foo bar quux" | MultiPart/mapper.py | sort -k1,1 | MultiPart/reducer.py 

reporter:counter:MyCounters,A,1
reporter:counter:MyCounters,A,1
reporter:counter:MyCounters,A,1
bar	0.14285714285714285
reporter:counter:MyCounters,A,1
reporter:counter:MyCounters,A,1
reporter:counter:MyCounters,B,1
foo	0.42857142857142855
reporter:counter:MyCounters,B,1
reporter:counter:MyCounters,C,1
labs	0.14285714285714285
reporter:counter:MyCounters,C,1
reporter:counter:MyCounters,C,1
reporter:counter:MyCounters,D,1
quux	0.2857142857142857
!total	1.0


In [36]:
# parts c - clear the output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/multipart-output
# NOTE: this directory won't exist unless you are re-running a job, that's fine.

rm: `/user/root/demo3/multipart-output': No such file or directory


In [ ]:
# part c - Hadoop streaming command - FILL IN HERE (don't forget to specify 4 reducers)







In [58]:
# <--- SOLUTION --->
# https://issues.apache.org/jira/browse/MAPREDUCE-3310 combiners don't play nice with secondary sort!
# part c - Hadoop streaming command
!hdfs dfs -rm -r {HDFS_DIR}/multipart-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1"  \
  -files MultiPart/mapper.py,MultiPart/combiner.py,MultiPart/reducer.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/alice.txt \
  -output {HDFS_DIR}/multipart-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4 

Deleted /user/root/demo3/multipart-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob3024830843428666992.jar tmpDir=null
21/01/23 17:46:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/23 17:46:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/23 17:46:12 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/23 17:46:12 INFO mapreduce.JobSubmitter: number of splits:2
21/01/23 17:46:12 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610967571082_0011
21/01/23 17:46:13 INFO impl.YarnClientImpl: Submitted application application_1610967571082_0011
21/01/23 17:46:13 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610967571082_0011/
21/01/23 17:46:13 INFO mapreduce.Job: Running job: job_1610967571082_0011
21/01/23 17:46:22 INFO mapreduce.Job: Job job_1610967571082_0011 running in uber mode : false
21/01/23 17:46:22 I

In [59]:
#  part c - take a look at a few records from each partition (RUN THIS CELL AS IS)
for p in range(4):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/multipart-output/part-0000{p} | head -n 5

========== PARTITION1 ==========
t	0.007119994774315762
table	0.0005878894767783657
tail	0.00029394473838918284
tails	9.798157946306095e-05
take	0.0007185315827291136
cat: Unable to write to output stream.
========== PARTITION2 ==========
a	0.022699065908942453
abide	6.532105297537396e-05
able	3.266052648768698e-05
about	0.003331373701744072
above	9.798157946306095e-05
========== PARTITION3 ==========
gained	3.266052648768698e-05
gallons	3.266052648768698e-05
game	0.0004245868443399308
games	3.266052648768698e-05
garden	0.0005225684238029917
========== PARTITION4 ==========
name	0.0003592657913645568
named	3.266052648768698e-05
names	6.532105297537396e-05
narrow	6.532105297537396e-05
nasty	3.266052648768698e-05
cat: Unable to write to output stream.


__Expected Results:__  
__`NOTE:`__ exact partition splits depend on your custom function:

Test file:
<table>
<th>partition 1</th>
<th>partition 2</th>
<th>partition 3</th>
<th>partition 4</th>
<tr>
<td><pre>
test	0.15
this	0.15
two	0.05
</pre></td>
<td><pre>
a	0.1
file	0.15
for	0.05
</pre></td>
<td><pre>
has	0.05
is	0.1
lines	0.05
</pre></td>
<td><pre>
small	0.15
</pre></td>
</tr></table>

Full Alice Text:
<table>
<tr>
<th>partition 1</th>
<th>partition 2</th>
</tr>
<tr>
<td><pre>
t	0.007119994774315762
table	0.0005878894767783657
tail	0.00029394473838918284
tails	9.798157946306095e-05
take	0.0007185315827291136
</pre></td>
<td><pre>
a	0.022699065908942453
abide	6.532105297537396e-05
able	3.266052648768698e-05
about	0.003331373701744072
above	9.798157946306095e-05
</pre></td>
</tr>
    <tr>
<th>partition 3</th>
<th>partition 4</th>
</tr>
    <tr><td><pre>
gained	3.266052648768698e-05
gallons	3.266052648768698e-05
game	0.0004245868443399308
games	3.266052648768698e-05
garden	0.0005225684238029917
</pre></td>
<td><pre>
name	0.0003592657913645568
named	3.266052648768698e-05
names	6.532105297537396e-05
narrow	6.532105297537396e-05
nasty	3.266052648768698e-05
</pre></td>
</tr></table>

>__DISCUSSION__
> * After adding the custom partition key what else did you have to do so that the order inversion pattern would still work?
> * In part `d` what did you see from your custom counters? Do these numbers match the number of keys in the result of each partition? Why/why not?
> * What other partitioning cuts did you try? What partitioning results in a really uneven split? what partitioning results in the most even split?
> * Do custom counters solve the load balancing challenge?
> * If we wanted to design a subsequent job to sort these results from highest to lowest frequency what partitioning strategy would you explore? Any particular challenge there? (__`HINT:`__ you may wish to consider the kinds of numbers you saw in the results from Exercise 1, how python stores floats, and what you know about word frequencies in natural language).

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after exercise 3)__
* After adding the custom partition key what else did you have to do so that the order inversion pattern would still work?
> We need to add a secondary sort.

* In part `d` what did you see from your custom counters? Do these numbers match the number of keys in the result of each partition? Why/why not?
> Results will vary for counts depending on the partition students choose & the ammount of combining. We saw: 
```
A=1391
B=845
C=1170
D=633 
```

 >Note that these are not the same as the number of resulting lines in each partition. Some words eg. 'the' will occur many more times than others, so even if we divide the alphabet evenly that may not be an even split of word's processed.
Use this opportunity to reinforce the fact that Hadoop did not put partition 'A' first & remind students about the hash function. They'll implement a Total Order Sort in HW2 where they reverse engineer that hash function to pre-order the partition keys.

* What other partitioning cuts did you try? What partitioning results in a really uneven split? what partitioning results in the most even split?
> Results will vary. Give students an opportunity to share & then discuss their reasoning.

* Do custom counters solve the load balancing challenge?
> No, they can help us determine whether we have a load balancing problem in the first place, but solving that problem if it exists requires some outside understading of our data distribution. That's usually one of the key goals behind our EDA. With really large datasets we'll also often do this EDA on a random sample instead of the full data -- we'll talk more about that in week 5.

* If we wanted to design a subsequent job to sort these results from highest to lowest frequency what partitioning strategy would you explore? Any particular challenge there? (__`HINT:`__ you may wish to consider the kinds of numbers you saw in the results from Exercise 1, how python stores floats, and what you know about word frequencies in natural language).
> We'd need to partition based on the relative frequencies but that is a challenge because the numbers are very very small and we're going to run into floating point errors. Another challenge is that this is going to be a inverse power law distribution --- lots of words which occur very few times & whose relative frequencies are clustered around 0. We could try a log-transformation to help us partition... but in practice we'd want to use EDA on specific words to design a partition strategy.

# Exercise 4: Unordered Total Sort vs Total Order Sort

In HW1 we emphasized that sorting can be a useful preprocessing tool because sorted input can sometimes facilitate a more efficient algorithm design. However sorting is also often desirable in its own right. For example suppose you wanted to get the top and bottom 100 words according to their relative frequencies? The best way to get these results would be to do a Total Order Sort -- that is to sort the entire data set from highest to lowest so that you can simply read the first 100 records in the first partition and last 100 records in the last partition.

So far we've tried two different ways of sorting using the Hadoop framework and neither quite achieved this result to satisfaction. First, in last week's breakout we performed a secondary sort using the `keycomparator` and accompanying Hadoop parameters. When we used a single reducer that strategy _did_ successfully result in a sort from top to bottom, but in a class all about scaling up for large data solutions that require a single reducer won't get us very far. Unfortunately when we tried using those same three sorting parameters and with multiple reducers we ended up with results sorted within each partition but not across partitions. This is what we call that a _partial sort_ because records are only ordered relative to the other keys that happen to be processed by the same reducer. Importantly a partial sort is of no use at all for us if we wanted to find the top 100 and bottom 100 from our relative frequencies file.

Then in Exercise 2 of this notebook we learned how to specify a custom partition key so that we can explicitly control the partitioning. By combining the use of a custom partition key and the secondary sort from last week we were able to ensure that our results were not only sorted within their paritions but also that the paritioning grouped records with similar value ranges together. In theory this should have solved our 'total sort with multiple reducers' challenge but in practice something odd happened: the records with the top value didn't reliably end up in the first partition nor did the records with the lowest values necessarily end up in the last. The partition files themselves were out of order, making this an _unordered total sort_.

The reason that the partitions appear out of order has to do with the hash function that Hadoop applies to your custom partition keys. That hash function results in an ordering that may not match the human readible string or integer numbering you as a programmer might have intended. Luckily, this hash function is both fixed and known -- so for example, if you are using partition keys `A`, `B`, and `C` your partitions will always end up ordered `B` - `C` -`A` (as you saw in Exercise 2). That means if we know in advance how many partitions (i.e. reducers) we plan to use, we can reverse engineer the hash function to figure out how Hadoop will order those keys and plan accordingly (for example in our 3 partition case by specifying that the top values should get the partition key `B` not `A`). When your data is sorted not only within each partition but the partitions are also in the right order, you've achieved a _total order sort_. 

Let's give it a try! Your job in this exercise is to write a Hadoop Job that will perform a total order sort on the output of Exercise 1 (relative frequencies) with any number of partitions.

__Note__: Part III.D.4 in the [Total Sort Notebook](https://github.com/UCB-w261/main/blob/master/HelpfulResources/TotalSortGuide/_total-sort-guide-hadoop-streaming.ipynb) has a much more comprehensive explanation that you may wish to reference here.


### Exercise 4 Tasks
* __a) discuss:__ In the `%%writefile` cell below we create a partition file (__`partitions.txt`__) which contains the cut points we'll use to partition the data. Based on your reading of this file, how many partitions will we use? Think about the kinds of values that we got in Exercise 1 ... can you see any potential problems with these cutpoints?


* __b) discuss:__ Read through __`TotalOrderSort/mapper.py`__ and __`TotalOrderSort/reducer.py`__. Which of these scripts makes use of our partition file? What does the mapper do? What does the reducer do? 

* __c) code:__ Run the provided code below to apply this mapper and reducer. Note how the Hadoop job reads directly from the output directory we created in Exercise 1 (you will need to be those results were computed inorder to run this job). Use the provided code to view the output and confirm that the current implementation performs an unordered sort. What adjustments might you want to make to the partition file? (go ahead and modify it as desired). Then modify the Hadoop job so that it uses `/bin/cat` instead of `reducer.py`... and re-run the job this will allow you to see the order in which the partition keys were sorted. 

* __d) discuss:__ Read through __`TotalOrderSort/TOS_mapper.py`__. Pay particular attention to the helper function `makeKeyHash()` -- what does this function do? how is it used? how is this mapper different that the original one? 

* __e) code + discussion:__ Replace the original mapper with this new mapper(_don't forget to add it to the `-files` line_) and rerun the job (still with `/bin/cat` as reducer). How did the parition ordering change? What happpens if you change the partition file so that it has one extra number? Does your job work? Re-place the true reducer. Et voila, you have achieved total order sort!

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS__
> __a)__ 5 partitions. Most relative frequencies will be very low decimals so most of the data is going to end up in the last few partitions. We can fix this by performing EDA on the input file and determining a way to balance the load across all 5 reducers. In this case we might use cutpoints more like: 0.5,0.3,0.2,0.1,0.

> __b)__ The mapper reads the partition file and uses those cut points to assign letters for partition keys (eg. A, B, C, etc). The reducer simply removes these partition keys.

> __d)__ `makeKeyHash` allows us to sort the parition keys in the order that Hadoop will, this different ordering of partition keys is the only difference between this mapper and the last.

In [47]:
%%writefile TotalOrderSort/partitions.txt
0.01,0.005,0.003,0.002,0.001,0

Overwriting TotalOrderSort/partitions.txt


In [24]:
# part c - make sure files are executable (RUN THIS CELL AS IS)
!chmod a+x TotalOrderSort/mapper.py
!chmod a+x TotalOrderSort/reducer.py
!chmod a+x TotalOrderSort/TOS_mapper.py

In [48]:
# part c - hadoop job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/tos-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -files TotalOrderSort/mapper.py,TotalOrderSort/reducer.py,TotalOrderSort/partitions.txt \
  -mapper mapper.py \
  -reducer reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/frequencies-output \
  -output {HDFS_DIR}/tos-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 6

Deleted /user/root/demo3/tos-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob3305256276489392893.jar tmpDir=null
18/09/19 07:41:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/19 07:41:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/19 07:41:24 INFO mapred.FileInputFormat: Total input paths to process : 1
18/09/19 07:41:24 INFO mapreduce.JobSubmitter: number of splits:2
18/09/19 07:41:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537278954088_0020
18/09/19 07:41:25 INFO impl.YarnClientImpl: Submitted application application_1537278954088_0020
18/09/19 07:41:25 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1537278954088_0020/
18/09/19 07:41:25 INFO mapreduce.Job: Running job: job_1537278954088_0020
18/09/19 07:41:33 INFO mapreduce.Job: Job job_1537278954088_0020 running in uber mode : false
18/09/19 07:41:33

In [52]:
# <--- SOLUTION --->
# part c - hadoop job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/tos-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -files TotalOrderSort/TOS_mapper.py,TotalOrderSort/reducer.py,TotalOrderSort/partitions.txt \
  -mapper TOS_mapper.py \
  -reducer reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/frequencies-output \
  -output {HDFS_DIR}/tos-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 6

Deleted /user/root/demo3/tos-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.0.jar] /tmp/streamjob3874368387786140947.jar tmpDir=null
18/09/19 07:50:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/19 07:50:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/19 07:50:14 INFO mapred.FileInputFormat: Total input paths to process : 1
18/09/19 07:50:14 INFO mapreduce.JobSubmitter: number of splits:2
18/09/19 07:50:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1537278954088_0022
18/09/19 07:50:14 INFO impl.YarnClientImpl: Submitted application application_1537278954088_0022
18/09/19 07:50:14 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1537278954088_0022/
18/09/19 07:50:14 INFO mapreduce.Job: Running job: job_1537278954088_0022
18/09/19 07:50:23 INFO mapreduce.Job: Job job_1537278954088_0022 running in uber mode : false
18/09/19 07:50:23

In [53]:
# part c - print top words in each class (RUN THIS CELL AS IS)
for idx in range(6):
    print(f"\n============== PART-0000{idx}===============")
    numLines = !hdfs dfs -cat {HDFS_DIR}/tos-output/part-0000{idx} | wc -l
    print(f"Number of lines processed by this reducer: {numLines[0]}\n")
    !hdfs dfs -cat {HDFS_DIR}/tos-output/part-0000{idx} | head | column -t


============== PART-00000===============
Number of lines processed by this reducer: 15

!total  1.0
the     0.059757420372744306
and     0.03089767610031884
to      0.026591723367189297
a       0.0226802090523617
of      0.020740886829043816
it      0.020050619597015415
she     0.018177037110081187
i       0.01791407816454656
you     0.015810406600269535

============== PART-00001===============
Number of lines processed by this reducer: 16

as    0.009006343884561023
her   0.00815172731157348
with  0.00749432994773691
at    0.007461460079545081
s     0.007198501134010452
t     0.0071656312658186245
on    0.006705453111133025
all   0.0065739736383657104
this  0.005949446142720968
for   0.005883706406337311

============== PART-00002===============
Number of lines processed by this reducer: 18

so      0.004996219965157939
very    0.0047661308878151395
what    0.004667521283239654
is      0.004437432205896854
he      0.004207343128554054
little  0.004207343128554054
out     0.003878644